# Entrega 2, Grupo 02 - Aprendizaje Bayesiano

- Santiago Alaniz,  5082647-6, santiago.alaniz@fing.edu.uy
- Bruno De Simone,  4914555-0, bruno.de.simone@fing.edu.uy
- María Usuca,      4891124-3, maria.usuca@fing.edu.uy

## Objetivo

Implementar un algoritmo de predicción de palabras utilizando Naive Bayes. El modelo se entrenará con datos de conversaciones de WhatsApp y se probará en un simulador de cliente. 

El algoritmo debe ser capaz de recomendar palabras basadas en las últimas N palabras ingresadas en una frase, donde N es un hiperparámetro que se variará para evaluar el desempeño del modelo. Además, el modelo se reentrenará al finalizar cada frase para adaptarse a nueva evidencia. 

La implementación debe ser eficiente en términos de uso de CPU, utilizando las estructuras de datos más adecuadas en Python.


## Diseño

### Carga del dump de conversaciones en un grupo de WhatsApp

En la letra del laboratorio se menciona que se puede acceder al dump en formato `.csv` pero esto es un error en la letra. *WhatsApp* exporta las conversaciones en formato `.txt`.

Esto complejiza la carga de los datos ya que no podemos asumir que cada linea del archivo es una entrada del log.

Detectamos el siguiente patron en el archivo de texto

```
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
    ***
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
```

Nos valemos del modulo `src.regex` para definir la expresion regular que detecta el patron y extrae los mensajes (`LOG_ENTRY_PATTERN`).

`LOG_ENTRY_PATTERN = f'{metadata_pattern}{message_pattern}(?=\n{metadata_pattern}|$)':`

Esta expresion regular captura el contenido de cada entrada del log en dos grupos: `metadata` y `message`. 

- El grupo `metadata` captura la fecha y hora de la entrada, el nombre del usuario o el numero de telefono. 
- El grupo `message` captura el mensaje en si mismo y es el que nos interesa recuperar.

In [ ]:
import re
from src.regex import LOG_ENTRY_PATTERN

FILE_PATH = './assets/chat.txt'
PATTERN = LOG_ENTRY_PATTERN

with open(FILE_PATH, 'r', encoding='utf-8') as file:
  chat = file.read()

matches = re.findall(PATTERN, chat)
data = [ match[1] for match in matches ]

print(f'Cantidad de mensajes: {len(data)}')
data[:10]

### Preprocesamiento de datos

Para un algoritmo de sugerencia de palabras basado en aprendizaje bayesiano, el preprocesamiento de los datos es crucial para obtener resultados significativos. En nuestro caso, se compone de los siguientes pasos:

1. **Conversión a minúsculas**: Para que el algoritmo no trate las palabras como diferentes solo debido a las diferencias de mayúsculas y minúsculas.
2. **Eliminación de caracteres especiales, numericos, emojis**: Estos componentes lexicos y semanticos no aportan información relevante para el modelo que busca sugerir palabras.
3. **Eliminación de [stopwords](https://es.wikipedia.org/wiki/Palabra_vac%C3%ADa)**: Estas palabras aparecen con mucha frecuencia en el lenguaje natural independientemente del contexto, por lo que no aportan información relevante para el modelo.
4. **Tokenización ordenada**: Para que el algoritmo pueda identificar correctamente las palabras y su posición en la oración, es necesario dividir la oración en [tokens](https://es.wikipedia.org/wiki/Token).
5. **Cruzamiento de palabras con un [corpus](http://universal.elra.info/product_info.php?cPath=42_43&products_id=1509)**: Para descartar palabras que no pertenecen al idioma español, chequeamos su existencia en un [corpus](https://es.wikipedia.org/wiki/Corpus_ling%C3%BC%C3%ADstico). Esto es importante para evitar que el modelo sugiera palabras en otros idiomas o palabras que no existen.
6. **Relajar tildes**: Para que el modelo no distinga palabras con tildes de palabras sin tildes. Teniendo en cuenta que en los mensajes de WhatsApp es muy común que se omitan las tildes.

Construimos la clase `src.G02Preprocessor` para encapsular el preprocesamiento de los datos y realizar los pasos mencionados anteriormente con mayor facilidad y control.

*Nota:* 

La lista resultado del preprocesamiento tiene un tamaño menor a la lista original. Esto se debe a que el preprocesamiento puede descartar mensajes que no cumplen con los criterios de preprocesamiento.

In [ ]:
from src.preprocessing import G02Preprocessor

preprocessor = G02Preprocessor()
preprocessed_data = preprocessor.apply(data)

print(f'Cantidad de mensajes post preprocesamiento: {len(preprocessed_data)}')
preprocessed_data[:10]

In [ ]:
import nltk
from nltk.corpus import words
from collections import Counter

TXT_PATH = './assets/chat.txt'

nltk.download('words')
spanish_words = set(words.words())

with open(TXT_PATH, 'r', encoding='utf-8') as file:
    lines = file.readlines()

filtered_content = [line.split(':', 2)[-1].strip() for line in lines]
all_words = ' '.join(filtered_content).split()
all_words_lower = [word.lower() for word in all_words]


filtered_words = [word for word in all_words_lower if word in spanish_words]

word_count = Counter(filtered_words)
sorted_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

for word, frequency in sorted_words:
    print(f"{word}: {frequency}")

Opcion usando un txt como diccionario. (en nltk no estan todas las palabras, por ejemplo, pollo)

### Algoritmo
Extensiones del algoritmo original necesarias para la resolución del problema: tratamiento de atributos faltantes, numéricos, etc. (si es el propio algoritmo el que lo maneja), implementaciones adicionales necesarias para manejar ensambles de clasificadores, etc.

### Evaluación
- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.

## Experimentación

### Simulador de cliente.

El experimento consiste en evaluar los distintos hiperparametros del modelo. Alterando el tamaño de la ventana de palabras, y el vocabulario buscamos encontrar un equilibrio entre las siguientes propiedades deseables:

- **Memoria**: El modelo debe ser capaz de sugerir palabras que tengan coherencia con el grupo de Whatsapp de donde se extrajeron los datos.

- **Adaptacion**: El modelo debe ser capaz de adaptarse a la nueva evidencia, y mejorar su performance, sugiriendo palabras que tengan coherencia con la sesion de chat que se esta simulando.

El siguiente script permite simular el comportamiento de un cliente que escribe frases, fue proporcionado por el cuerpo docente. Es de utilidad para evaluar el desempeño del modelo, ya que permite ingresar frases y ver las sugerencias que el modelo realiza.


In [ ]:
def recomendacion_bayesiana(frase):
  import random

  dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

  # ## PSEUDOCODIGO - pag 55 - diapo##

  # D = ["mi", "hijo", "se", "olvidó", "de", "la"]
  # Horizonte = 4 #hiperparametro
  # h_MAP = ""
  # p_MAP = 0
  # for h in P:
    
  #   prob = P[h]
    
  #   for d in D[-Horizonte:]:
  #     prob = prob * PD[h].get(d, P_nada) 
  #     #P_nada es un valor que debemos definir para el caso cuando la palabra no se encuentre en el listado
    
  #   if prob > p_MAP:
  #     h_MAP , p_MAP = h , prob
  # print(h_MAP)

  return(random.choice(dias))


##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""

while 1:
  palabra = input(">> ")

  if palabra == "..":
    break

  elif palabra == ".":
    print("----- Comenzando frase nueva -----")
    frase = []

  elif palabra == "": # acepta última palabra sugerida
    frase.append(palabra_sugerida)

  else: # escribió una palabra
    frase.append(palabra)

  if frase:
    palabra_sugerida = recomendacion_bayesiana(frase)

    frase_propuesta = frase.copy()
    frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

    print(" ".join(frase_propuesta))

## Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?